In [0]:
# ==============================================================================
# NOTEBOOK DE INFERÊNCIA - PREVISÃO DE PARTIDAS FUTURAS
# ==============================================================================
from pyspark.ml import PipelineModel
from pyspark.sql import functions as F
from pyspark.ml.feature import IndexToString, VectorAssembler
import mlflow
import os

print("="*80)
print("PIPELINE DE INFERÊNCIA - PREVISÃO BRASILEIRÃO")
print("="*80)

spark.sql("USE CATALOG previsao_brasileirao")

# ==============================================================================
# PASSO 1: CONFIGURAR E CARREGAR MODELOS
# ==============================================================================
print("\n[PASSO 1] Configurando ambiente e carregando modelos...")
print("-"*80)

schema_modelos = "diamond"

# CONFIGURAÇÃO CRÍTICA: Volume path para Databricks Serverless
volume_path = "/Volumes/previsao_brasileirao/diamond/mlflow_models"
os.environ['MLFLOW_DFS_TMP'] = volume_path

print(f"📁 Volume configurado: {volume_path}")

# Configurar MLflow
mlflow.set_registry_uri("databricks-uc")

# Carregar metadados dos modelos
df_modelos_info = spark.table(f"{schema_modelos}.modelos_registry") \
    .filter("ativo = true") \
    .orderBy(F.desc("versao"))

print("\n📊 Modelos disponíveis:")
display(df_modelos_info.select("nome_modelo", "tipo", "acuracia", "run_id"))

# Pegar o run_id mais recente
run_id = df_modelos_info.select("run_id").first()["run_id"]
print(f"\n✅ Usando Run ID: {run_id}")

# Carregar os modelos
print("\nCarregando modelos treinados...")

try:
    modelo_geral = mlflow.spark.load_model(
        f"runs:/{run_id}/modelo_geral",
        dfs_tmpdir=volume_path
    )
    print("  ✅ Modelo Geral carregado")
    
    modelo_lr = mlflow.spark.load_model(
        f"runs:/{run_id}/modelo_lr",
        dfs_tmpdir=volume_path
    )
    print("  ✅ Modelo LR carregado")
    
    modelo_rf = mlflow.spark.load_model(
        f"runs:/{run_id}/modelo_rf",
        dfs_tmpdir=volume_path
    )
    print("  ✅ Modelo RF carregado")
    
    modelo_final = mlflow.spark.load_model(
        f"runs:/{run_id}/modelo_final",
        dfs_tmpdir=volume_path
    )
    print("  ✅ Meta-Modelo carregado")
    
except Exception as e:
    print(f"\n❌ ERRO ao carregar modelos: {e}")
    print("\n🔧 Possíveis soluções:")
    print("1. Verifique se o notebook '04_Diamond_Model_Training' foi executado")
    print("2. Execute novamente o PASSO 5 do notebook de treinamento")
    raise

# Carregar mapa de labels
df_labels_map = spark.table(f"{schema_modelos}.label_mapping")
label_mapping_dict = {row['resultado']: row['label'] for row in df_labels_map.collect()}
labels_array = [k for k, v in sorted(label_mapping_dict.items(), key=lambda x: x[1])]

print(f"\n✅ Labels mapeados: {labels_array}")

# Recriar transformadores
assembler_meta = VectorAssembler(
    inputCols=["prob_geral", "prob_lr", "prob_rf"],
    outputCol="features"
)

converter = IndexToString(
    inputCol="prediction",
    outputCol="previsao_texto",
    labels=labels_array
)

print("  ✅ Transformadores criados")

# Carregar dados auxiliares
df_partidas_historico = spark.table("gold.feature_store")
df_clubes = spark.table("silver.clubes")

print(f"\n✅ Dados históricos: {df_partidas_historico.count()} partidas")
print(f"✅ Clubes cadastrados: {df_clubes.count()}")

print("\n" + "="*80)
print("✅ PASSO 1 CONCLUÍDO - Modelos carregados com sucesso!")
print("="*80)

# ==============================================================================
# PASSO 2: CARREGAR E PREPARAR PARTIDAS FUTURAS
# ==============================================================================
print("\n[PASSO 2] Carregando partidas futuras...")
print("-"*80)

df_futuro_raw = spark.table("bronze.partidas_raw") \
    .filter("placar_oficial_mandante IS NULL AND valida = true") \
    .orderBy("partida_data")

num_partidas_futuras = df_futuro_raw.count()
print(f"\n📋 Partidas futuras encontradas: {num_partidas_futuras}")

if num_partidas_futuras == 0:
    print("\n⚠️ ATENÇÃO: Nenhuma partida futura encontrada!")
    print("\n💡 Possíveis motivos:")
    print("  - A rodada atual já foi concluída")
    print("  - A próxima rodada ainda não foi disponibilizada pela API")
    print("\n🔧 Solução:")
    print("  Execute o notebook '01_Bronze_Ingestao' para atualizar os dados")
    
    # Ainda assim, continuar para mostrar o processo
    print("\n  (Continuando com o processo para demonstração...)")
else:
    print("\n✅ Preview das partidas futuras:")
    display(df_futuro_raw.select(
        "partida_id", "rodada", "partida_data",
        "clube_casa_id", "clube_visitante_id"
    ).limit(5))

# ==============================================================================
# PASSO 3: ENGENHARIA DE FEATURES PARA PARTIDAS FUTURAS
# ==============================================================================
print("\n[PASSO 3] Aplicando engenharia de features...")
print("-"*80)

# Pegar Elo mais recente de cada time
latest_elos = df_partidas_historico \
    .groupBy("mandante_id") \
    .agg(F.max("rodada").alias("rodada")) \
    .join(df_partidas_historico, ["mandante_id", "rodada"]) \
    .select("mandante_id", F.col("elo_mandante_pre_jogo").alias("elo_atual"))

# Pegar médias móveis mais recentes
latest_mms_m = df_partidas_historico \
    .groupBy("mandante_id") \
    .agg(F.max("rodada").alias("rodada")) \
    .join(df_partidas_historico, ["mandante_id", "rodada"]) \
    .select("mandante_id", "mm_gols_m", "mm_fin_m", "mm_des_m")

latest_mms_v = df_partidas_historico \
    .groupBy("visitante_id") \
    .agg(F.max("rodada").alias("rodada")) \
    .join(df_partidas_historico, ["visitante_id", "rodada"]) \
    .select("visitante_id", "mm_gols_v", "mm_fin_v", "mm_des_v")

# Preparar partidas futuras
df_futuro_features = df_futuro_raw.select(
    F.col("partida_id"),
    F.col("rodada"),
    F.col("clube_casa_id").alias("mandante_id"),
    F.col("clube_visitante_id").alias("visitante_id"),
    F.to_timestamp("partida_data").alias("data_partida")
)

# Juntar Elo do mandante
df_com_elo_m = df_futuro_features.join(
    latest_elos,
    df_futuro_features["mandante_id"] == latest_elos["mandante_id"],
    "left"
).withColumnRenamed("elo_atual", "elo_mandante_pre_jogo") \
 .drop(latest_elos["mandante_id"])

# Juntar Elo do visitante
df_com_elo_mv = df_com_elo_m.join(
    latest_elos,
    df_com_elo_m["visitante_id"] == latest_elos["mandante_id"],
    "left"
).withColumnRenamed("elo_atual", "elo_visitante_pre_jogo") \
 .drop(latest_elos["mandante_id"])

# Juntar médias móveis do mandante
df_com_mm_m = df_com_elo_mv.join(
    latest_mms_m,
    df_com_elo_mv["mandante_id"] == latest_mms_m["mandante_id"],
    "left"
).drop(latest_mms_m["mandante_id"])

# Juntar médias móveis do visitante
df_com_tudo = df_com_mm_m.join(
    latest_mms_v,
    df_com_mm_m["visitante_id"] == latest_mms_v["visitante_id"],
    "left"
).drop(latest_mms_v["visitante_id"])

# Calcular diferenciais
df_futuro_features_final = df_com_tudo \
    .withColumn("elo_diff", F.col("elo_mandante_pre_jogo") - F.col("elo_visitante_pre_jogo")) \
    .withColumn("diff_mm_gols", F.col("mm_gols_m") - F.col("mm_gols_v")) \
    .withColumn("diff_mm_fin", F.col("mm_fin_m") - F.col("mm_fin_v")) \
    .withColumn("diff_mm_des", F.col("mm_des_m") - F.col("mm_des_v")) \
    .na.fill(0)

print("  ✅ Features calculadas")

print("\n" + "="*80)
print("✅ PASSO 3 CONCLUÍDO - Features prontas!")
print("="*80)

# ==============================================================================
# PASSO 4: FAZER PREVISÕES
# ==============================================================================
print("\n[PASSO 4] Gerando previsões...")
print("-"*80)

if num_partidas_futuras > 0:
    # Nível 1: Previsões dos especialistas
    print("\n  Nível 1: Modelos Especialistas...")
    pred_geral_futuro = modelo_geral.transform(df_futuro_features_final)
    pred_lr_futuro = modelo_lr.transform(df_futuro_features_final)
    pred_rf_futuro = modelo_rf.transform(df_futuro_features_final)
    print("    ✅ Previsões dos especialistas geradas")
    
    # Preparar meta-features
    meta_features_geral = pred_geral_futuro.select("partida_id", F.col("probability").alias("prob_geral"))
    meta_features_lr = pred_lr_futuro.select("partida_id", F.col("probability").alias("prob_lr"))
    meta_features_rf = pred_rf_futuro.select("partida_id", F.col("probability").alias("prob_rf"))
    
    df_inferencia_meta = df_futuro_features_final.select("partida_id") \
        .join(meta_features_geral, "partida_id") \
        .join(meta_features_lr, "partida_id") \
        .join(meta_features_rf, "partida_id")
    
    df_inferencia_meta = assembler_meta.transform(df_inferencia_meta)
    
    # Nível 2: Meta-modelo
    print("  Nível 2: Meta-Modelo...")
    previsoes_finais = modelo_final.transform(df_inferencia_meta)
    print("    ✅ Previsões finais geradas")
    
    # Converter índices para texto
    previsoes_texto = converter.transform(previsoes_finais)
    
    # Juntar com nomes dos clubes
    df_resultado_final = df_futuro_features_final \
        .join(previsoes_texto.select("partida_id", "previsao_texto", "probability"), "partida_id") \
        .join(df_clubes.alias("mandante"), F.col("mandante_id") == F.col("mandante.clube_id")) \
        .join(df_clubes.alias("visitante"), F.col("visitante_id") == F.col("visitante.clube_id")) \
        .withColumn("competicao", F.lit("Brasileirão Série A")) \
        .select(
            "competicao",
            F.date_format("data_partida", "dd/MM/yyyy HH:mm").alias("data_hora_partida"),
            F.col("mandante.nome").alias("time_mandante"),
            F.col("visitante.nome").alias("time_visitante"),
            F.col("previsao_texto").alias("previsao"),
            F.col("probability").alias("confianca_probabilidades")
        ).orderBy("data_hora_partida")
    
    # Salvar na camada Diamond
    df_resultado_final.write \
        .mode("overwrite") \
        .format("delta") \
        .saveAsTable("diamond.previsoes_proximas_partidas")
    
    print("\n  ✅ Previsões salvas em 'diamond.previsoes_proximas_partidas'")
    
    # Criar view temporária
    df_resultado_final.createOrReplaceTempView("visualizacao_previsoes")
    
    print("\n" + "="*80)
    print("✅ PASSO 4 CONCLUÍDO - Previsões geradas!")
    print("="*80)
    
    # ==============================================================================
    # VISUALIZAÇÃO FINAL
    # ==============================================================================
    print("\n" + "="*80)
    print("📊 PREVISÕES DAS PRÓXIMAS PARTIDAS")
    print("="*80)
    
    display(df_resultado_final)
    
    print("\n✅ Você pode consultar as previsões usando:")
    print("   SELECT * FROM diamond.previsoes_proximas_partidas")
    print("   ou")
    print("   SELECT * FROM visualizacao_previsoes")
    
else:
    print("\n⚠️ Nenhuma previsão gerada pois não há partidas futuras")
    print("   Execute o notebook '01_Bronze_Ingestao' e tente novamente")

print("\n" + "="*80)
print("✅✅✅ PIPELINE DE INFERÊNCIA CONCLUÍDO! ✅✅✅")
print("="*80)